In [1]:
#Nama : Muhammad Rizal Muhaimin

#Persiapan Awal

import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#menampilkan versi tf
print(tf.__version__)

In [ ]:
#mengmbil data
!wget --no-check-certificate \
https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
-O /tmp/rockpaperscissors.zip

In [ ]:
# melakukan ekstraksi pada file zip
import zipfile,os
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r') # mode read
zip_ref.extractall('/tmp')
zip_ref.close() 

In [ ]:
#melihat alamat data 
os.listdir('/tmp')

In [ ]:
os.listdir('/tmp/rockpaperscissors')

In [ ]:
#Membaca petunjuk data
baca_Readme = open('/tmp/rockpaperscissors/README_rpc-cv-images.txt','r')
Text = baca_Readme.readlines()

for  line in Text:
  print(line)
baca_Readme.close()

In [ ]:
#Merapikan analisis data
my_base_dir = '/tmp/rockpaperscissors/rps-cv-images'
file_name = []
tag = []
full_path = []
for path, subdirs, files in os.walk(my_base_dir):
    for name in files:
        full_path.append(os.path.join(path, name)) 
        tag.append(path.split('/')[-1])        
        file_name.append(name)

In [ ]:

import pandas as pd
# memasukan variabel ke sebuah dataframe agar rapih
df = pd.DataFrame({"path":full_path,'file_name':file_name,"tag":tag})
df.groupby(['tag']).size()

In [ ]:
#dari data diatas ternyata antara keterangan readme dan jumlah data berbeda
#akan tetapi jumlah data sesuai dengan keterangan dicoding
#cek sample datanya
df.head()

In [ ]:
#Memperiapkan data input dan label
X= df['path']
y= df['tag']

In [ ]:
#menghilangkan label dengan nilai 'rps-cv-images'

X = X[1:]
y = y[1:]

In [ ]:
print("Jumlah data_X:",len(X))
print("Jumlah data_Y:",len(y))

In [ ]:
#load library untuk train test split
from sklearn.model_selection import train_test_split

In [ ]:
# split dataset awal menjadi data train dan test
X_train, X_validation, y_train, y_validation = train_test_split(
    X, y, test_size=0.2, random_state=100)

In [ ]:
#memastikan sesuai spesifikasi
print("Jumlah data_X_train:",len(X_train))
print("Jumlah data_X_validation:",len(X_validation))

In [ ]:
# menyatukan  dataframe

df_tr = pd.DataFrame({'path':X_train
              ,'tag':y_train
             ,'set':'train'})

df_val = pd.DataFrame({'path':X_validation
              ,'tag':y_validation
             ,'set':'validation'})

In [ ]:
# melihat proporsi pada masing masing set 
df_all = df_tr.append([df_tr,df_val]).reset_index(drop=1)
print(df_all.groupby(['set','tag']).size(),'\n')
df_all.sample(3)

In [ ]:
#Persiapan perapian dan pembuatan file dataset
import shutil
from tqdm.notebook import tqdm as tq

In [ ]:
datasource_path = '/tmp/rockpaperscissors/rps-cv-images'
dataset_path = "dataset/"

In [ ]:
for index, row in tq(df_all.iterrows()):
    
    #detect filepath
    file_path = row['path']
    if os.path.exists(file_path) == False:
            file_path = os.path.join(datasource_path,row['tag'],row['image'].split('.')[0])            
    
    #make folder destination dirs
    if os.path.exists(os.path.join(dataset_path,row['set'],row['tag'])) == False:
        os.makedirs(os.path.join(dataset_path,row['set'],row['tag']))
    
    #define file dest
    destination_file_name = file_path.split('/')[-1]
    file_dest = os.path.join(dataset_path,row['set'],row['tag'],destination_file_name)
    
    #copy file from source to dest
    if os.path.exists(file_dest) == False:
        shutil.copy2(file_path,file_dest)

In [ ]:
#Memastikan dataset terbentuk
os.listdir("dataset/")

In [ ]:
os.listdir("dataset/train")

In [ ]:
base_dir = 'dataset/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [ ]:
# rock direktori data training
train_rock_dir = os.path.join(train_dir, 'rock')

# scissors direktori data training
train_scissors_dir = os.path.join(train_dir, 'scissors')

# paper direktori data training
train_paper_dir = os.path.join(train_dir, 'paper')

# rock direktori data validasi
train_rock_dir = os.path.join(validation_dir, 'rock')

# scissors direktori data validasi
train_scissors_dir = os.path.join(validation_dir, 'scissors')

# paper direktori data validasi
train_paper_dir = os.path.join(validation_dir, 'paper')


In [ ]:
#persiapan GenerateImage
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(150, 150),  
        batch_size=32,
        # karena kita merupakan masalah klasifikasi 3 kelas maka menggunakan class_mode = 'categorical'
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir, # direktori data validasi
        target_size=(150, 150),
        batch_size=32, 
        # karena kita merupakan masalah klasifikasi 3 kelas maka menggunakan class_mode = 'categorical'
        class_mode='categorical')

In [ ]:
#optimalisasi dengan tf.data
def tf_data_generator(generator, input_shape):
    num_class = generator.num_classes
    tf_generator = tf.data.Dataset.from_generator(
        lambda: generator,
        output_types=(tf.float32, tf.float32),
        output_shapes=([None
                        , input_shape[0]
                        , input_shape[1]
                        , input_shape[2]]
                       ,[None, num_class])
    )
    return tf_generator

    
train_data = tf_data_generator(train_generator, (150,150,3))

validation_data = tf_data_generator(validation_generator, (150,150,3))

In [ ]:
#Pembuatan model TF
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='sigmoid')
 ])

In [ ]:
#kompail data yang dpakai
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])


In [ ]:
#fit data dengan model
Data_fit = model.fit(
      train_generator,
      steps_per_epoch=25,  # berapa batch yang akan dieksekusi pada setiap epoch
      epochs=25,
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=5,  # erapa batch yang akan dieksekusi pada setiap epoch
      shuffle=True,
      verbose=2)

In [ ]:
#melihat hasil test data
test_loss, test_acc = model.evaluate(train_data,  steps=25, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
#visualisasi hasil fit
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
 

acc = Data_fit.history['accuracy']
val_acc = Data_fit.history['val_accuracy']

loss = Data_fit.history['loss']
val_loss = Data_fit.history['val_loss']

epochs_range = range(25)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#Prediksi gambar masukan dari user
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

uploaded = files.upload()

# Parameters
input_size = (150,150) # Bisa ganti sesuai kebutuhan
#define labels
labels = ['peper', 'rock', 'scissors']

def preprocess(img,input_size):
    nimg = img.convert('RGB').resize(input_size, resample= 0)
    img_arr = (np.array(nimg))/255
    return img_arr

def reshape(imgs_arr):
    return np.stack(imgs_arr, axis=0)

for fn in uploaded.keys():

  path = fn 
  img = image.load_img(path)
  img1 = image.load_img(path, target_size=(150,150))
 
  # read image
  X = preprocess(img,input_size)
  X = reshape([X])
  y = model.predict(X)
  #menaampilkan gambar
  imgplot = plt.imshow(img1)
  #hasil prediksi
  print( labels[np.argmax(y)], np.max(y) )
  